导入数据

In [6]:
import pandas as pd

# data=pd.read_csv('data/上海2019_2023年按月分类文件/20191201_20200101 57356 条.csv')[:50]
data=pd.read_csv(r'data/上海2019_2023年按月分类文件_情绪值/百度+腾讯的对比验证/20190301_20190401 26184 条_Baidu&Tencent&BERT&Ali 21747 条.csv')
print(len(data)) #数据的行数
data.head(5)

21747


,mid,lat,lng,user_name,user_link,verify_typ,weibo_link,publish_time,content,image_urls,...,Tencent_positive,Tencent_neutral,Tencent_negative,Tencent_sentiment,BERT_ad_prob,BERT_pred,ali_positive,ali_sentiment,ali_neutral,ali_negative
0,4.344820e+15,31.216145,121.412377,小心獭兔,https://weibo.com/312247111,没有认证,https://weibo.com/2188120701/HiO2wcvEF,2019/3/1 0:04,3月要快乐就要跟她学马路舞。跳完过马路会平平安安。真的很可爱。哈哈哈哈哈。#搞笑视频# L...,NaN,...,0.654513,0.330944,0.014543,positive,0.998641,1,NaN,NaN,NaN,NaN
1,4.344820e+15,31.198070,121.519220,JsWyhla,https://weibo.com/u/2964873937,没有认证,https://weibo.com/2964873937/HiO349Wt6,2019/3/1 0:06,忙碌的一天 2上海·上海东方电视台 ​​​​,https://wx1.sinaimg.cn/large/b0b862d1gy1g0ml4a...,...,0.008661,0.971560,0.019779,neutral,0.534058,1,0.4453,负面,0.0164,0.5382
2,4.344820e+15,31.201960,121.444389,订位滴滴,https://weibo.com/5855499249,没有认证,https://weibo.com/5855499249/HiO3OgAJA,2019/3/1 0:08,我点评了2上海·衡山路： 2上海·衡山路,NaN,...,0.031652,0.955590,0.012758,neutral,0.456147,0,0.3252,负面,0.0217,0.6529
3,4.344820e+15,31.342420,121.492600,爱睡懒觉的黑喵,https://weibo.com/u/1009063544,没有认证,https://weibo.com/1009063544/HiO5jCXHZ,2019/3/1 0:11,#假面骑士龙骑# 那做个交易嘛～给我张生存卡就行了，我也不要别人的卡盒了～ 2上海·长宏新苑...,https://wx3.sinaimg.cn/large/3c251678gy1g0ml9p...,...,0.001443,0.948951,0.049606,neutral,0.681232,1,0.9913,正面,0.0003,0.0084
4,4.344820e+15,31.237490,121.484610,山东诚之旅徐娟,https://weibo.com/2107045807,黄V认证,https://weibo.com/2107045807/HiO5Laoig,2019/3/1 0:12,出差上海超话，谁说上海人超话饭量小，瞧瞧这碗，一份鱼，一份汤圆，撑坏了上海美食超话#上...,https://wx2.sinaimg.cn/large/7d96f7afly1g0mimb...,...,0.012902,0.773490,0.213608,neutral,0.990154,1,0.8974,正面,0.0176,0.0849


阿里云NLP的情感倾向分析

In [7]:
import json
import os

# 这里以分词为例，其它算法的API名称和参数请参考文档
from aliyunsdkalinlp.request.v20200629 import GetWsCustomizedChGeneralRequest
# 调用阿里的NLP
from aliyunsdkalinlp.request.v20200629 import GetSaChGeneralRequest

from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException

'''
 * 阿里云账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM用户进行API访问或日常运维，请登录RAM控制台创建RAM用户。
 * 此处以把AccessKey和AccessKeySecret保存在环境变量为例说明。您也可以根据业务需要，保存到配置文件里。
 * 强烈建议不要把AccessKey和AccessKeySecret保存到代码里，会存在密钥泄漏风险
'''

# access_key_id = os.environ['NLP_AK_ENV']
# access_key_secret = os.environ['NLP_SK_ENV']
import pandas as pd

key = pd.read_csv('data\key\AliAccessKey.csv')
# Key的读取  读取AccessKey和AccessKeySecret  
access_key_id = key['AccessKey ID'][0]
access_key_secret = key['AccessKey Secret'][0]

def ali_nlp(text):
    # 创建AcsClient实例
    client = AcsClient(
        access_key_id,
        access_key_secret,
        "cn-hangzhou"
    )
    request=GetSaChGeneralRequest.GetSaChGeneralRequest()
    # request.set_Text("云麦好轻智能体脂秤精准蓝牙秤体重测人体脂肪秤家用称健康电子秤")
    request.set_Text(text)
    request.set_ServiceCode("alinlp")
    response = client.do_action_with_exception(request)

    resp_obj = json.loads(response)

    # print(resp_obj)
    # print(resp_obj['Data'])

    jd=json.loads(resp_obj['Data'])
    # print(jd)

    # 提取所需的值
    positive_prob = jd['result']['positive_prob']
    sentiment = jd['result']['sentiment']
    neutral_prob = jd['result']['neutral_prob']
    negative_prob = jd['result']['negative_prob']


    return positive_prob, sentiment, neutral_prob, negative_prob

In [8]:
text="你好, 我是一名学生, 我很开心, 你呢？"

pos,sen,neu,neg=ali_nlp(text)
print("positive_prob",pos,"\nsentiment",sen,"\nneutral_prob",neu,"\nnegative_prob",neg)

positive_prob 0.2533 
sentiment 中性 
neutral_prob 0.7083 
negative_prob 0.0381


In [9]:
import pandas as pd
import re
import jieba
import numpy as np
import datetime
import csv
import time
from tqdm import tqdm

def clean_content(content,place):
    # 去除地名
    content = content.replace(place, '')
    # 去除一些关键词
    content =content.replace('分享图片', '').replace('分享视频', '').replace('微博视频', '').replace('的微博视频', '').replace('网页链接','').replace('超话','').replace('新浪图片','').replace('<br>','')    
    # 去除英文
    content = re.sub(r'[a-zA-Z]+', '', content)
    content = re.sub(r'\d+', '', content).replace(' ', '').replace('.', '').replace('_','')
    # 去除所有非中文符号
    content = re.sub(r'[^\u4e00-\u9fa5]', '', content)
    # 去除空白字符
    content = re.sub(r'\s+', '', content)
    return content

# def clean_content(content,place):
#     # 去除地名
#     content = content.replace(place, '')
#     # 去除一些关键词
#     content =content.replace('分享图片', '').replace('分享视频', '').replace('微博视频', '').replace('的微博视频', '').replace('网页链接','').replace('超话','').replace('新浪图片','').replace('<br>','')
#     # 去除两个#和他们之间的文字
#     content = re.sub(r'#.*#', '', content)
#     # 去除@和他们之间的文字
#     content = re.sub(r'@.*@', '', content)
#     # 去除[]和他们之间的文字
#     content = re.sub(r'\[.*?\]', '', content)
#     # 去除<>和他们之间的文字
#     content = re.sub(r'<.*?>', '', content)
    
#     # 去除英文
#     content = re.sub(r'[a-zA-Z]+', '', content)
#     content = re.sub(r'\d+', '', content).replace(' ', '').replace('.', '').replace('_','')
#     # 去除所有非中文符号
#     content = re.sub(r'[^\u4e00-\u9fa5]', '', content)
#     # 去除空白字符
#     content = re.sub(r'\s+', '', content)
#     return content

time_start=time.time()

# # 对content列进行清洗 由对应的地名在文本中去除content文本中的内容
# data['clean_content'] = data.apply(lambda row: clean_content(row['content'], row['content_location_name']), axis=1)
# print("清洗完成")

# # 如果文本为空 去掉该行
# data = data[data['clean_content'] != '']
# print("去除空白行完成")

# # 保留字数大于3的
# data=data[data['clean_content'].apply(lambda x: len(x)>3)]
# print("保留字数大于3完成")

# # 对清洗后的文本 进行 分词 放入jieba_cut列中
# data['jieba_cut'] = data['clean_content'].apply(lambda x: jieba.lcut(x))
# print("分词完成")

# def process_row(row):
#     max_retries = 5  # 设置最大重试次数
#     retry_delay = 0.1  # 每次重试之间的等待时间（秒）
#     retries = 0
#     while retries < max_retries:
#         try:
#             positive_prob, sentiment, neutral_prob, negative_prob = ali_nlp(row['clean_content'])
#             if positive_prob is None or sentiment is None or neutral_prob is None or negative_prob is None:
#                 retries+=1
#                 time.sleep(retry_delay)
#                 continue
#             else: 
#                 return positive_prob, sentiment, neutral_prob, negative_prob
#         except:
#             retries+=1
#             time.sleep(retry_delay)
        
def process_row(row):
    try:
        if pd.isna(row['ali_positive']):
            positive_prob, sentiment, neutral_prob, negative_prob=ali_nlp(row['clean_content'])
            return positive_prob, sentiment, neutral_prob, negative_prob
        else:
            return row['ali_positive'], row['ali_sentiment'], row['ali_neutral'], row['ali_negative']
    except:
        print(row)
        print(row['clean_content'])
        print("阿里API调用失败")
        return np.nan,np.nan,np.nan,np.nan

# 使用apply返回的结果创建新的列
# results=[]
# for row in tqdm(data.itertuples(index=False)):
#     results.append(process_row(row))
tqdm.pandas()
results = data.progress_apply(process_row, axis=1)
# results = data.apply(process_row, axis=1)
data['ali_positive'], data['ali_sentiment'], data['ali_neutral'], data['ali_negative'] = zip(*results)
print("情感分析完成")

# 保存
# filename = 'data\WeiboTrainData\sentiment_analysis_data(Ali)_{}_5.csv'.format(str(datetime.datetime.now()).split(' ')[0])
filename='data/上海2019_2023年按月分类文件_情绪值/百度+腾讯的对比验证/20190301_20190401 26184 条_Baidu&Tencent&BERT&Ali 21747 条.csv'
with open(filename, 'w', encoding='utf-8-sig', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(data.columns)  # 写入列名
    for row in data.itertuples(index=False):
        writer.writerow(row)

time_end=time.time()
print(filename,"本次运行总共耗时：",time_end-time_start)
data.head(5)


100%|██████████| 21747/21747 [00:01<00:00, 17153.39it/s]


情感分析完成
data/上海2019_2023年按月分类文件_情绪值/百度+腾讯的对比验证/20190301_20190401 26184 条_Baidu&Tencent&BERT&Ali 21747 条.csv 本次运行总共耗时： 2.0192108154296875


,mid,lat,lng,user_name,user_link,verify_typ,weibo_link,publish_time,content,image_urls,...,Tencent_positive,Tencent_neutral,Tencent_negative,Tencent_sentiment,BERT_ad_prob,BERT_pred,ali_positive,ali_sentiment,ali_neutral,ali_negative
0,4.344820e+15,31.216145,121.412377,小心獭兔,https://weibo.com/312247111,没有认证,https://weibo.com/2188120701/HiO2wcvEF,2019/3/1 0:04,3月要快乐就要跟她学马路舞。跳完过马路会平平安安。真的很可爱。哈哈哈哈哈。#搞笑视频# L...,NaN,...,0.654513,0.330944,0.014543,positive,0.998641,1,0.9982,正面,0.0000,0.0017
1,4.344820e+15,31.198070,121.519220,JsWyhla,https://weibo.com/u/2964873937,没有认证,https://weibo.com/2964873937/HiO349Wt6,2019/3/1 0:06,忙碌的一天 2上海·上海东方电视台 ​​​​,https://wx1.sinaimg.cn/large/b0b862d1gy1g0ml4a...,...,0.008661,0.971560,0.019779,neutral,0.534058,1,0.4453,负面,0.0164,0.5382
2,4.344820e+15,31.201960,121.444389,订位滴滴,https://weibo.com/5855499249,没有认证,https://weibo.com/5855499249/HiO3OgAJA,2019/3/1 0:08,我点评了2上海·衡山路： 2上海·衡山路,NaN,...,0.031652,0.955590,0.012758,neutral,0.456147,0,0.3252,负面,0.0217,0.6529
3,4.344820e+15,31.342420,121.492600,爱睡懒觉的黑喵,https://weibo.com/u/1009063544,没有认证,https://weibo.com/1009063544/HiO5jCXHZ,2019/3/1 0:11,#假面骑士龙骑# 那做个交易嘛～给我张生存卡就行了，我也不要别人的卡盒了～ 2上海·长宏新苑...,https://wx3.sinaimg.cn/large/3c251678gy1g0ml9p...,...,0.001443,0.948951,0.049606,neutral,0.681232,1,0.9913,正面,0.0003,0.0084
4,4.344820e+15,31.237490,121.484610,山东诚之旅徐娟,https://weibo.com/2107045807,黄V认证,https://weibo.com/2107045807/HiO5Laoig,2019/3/1 0:12,出差上海超话，谁说上海人超话饭量小，瞧瞧这碗，一份鱼，一份汤圆，撑坏了上海美食超话#上...,https://wx2.sinaimg.cn/large/7d96f7afly1g0mimb...,...,0.012902,0.773490,0.213608,neutral,0.990154,1,0.8974,正面,0.0176,0.0849
